In [1]:
import pandas as pd
import re
import unidecode
import demoji
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
import spacy
from string import punctuation
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import tokenize
import nltk

%matplotlib inline

nlp = spacy.load("pt")
nlp.Defaults.stop_words |= {"to","uol","mi","budddhetg","the", "ne", "vou", "ta", "via","ex", "pq", "vc","aa","pra","to", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"}

In [2]:
palavras_irrelevantes = set(stopwords.words('portuguese') + list(punctuation))

In [3]:
data = pd.read_csv("data/tweeterLeo/dataTweeter.csv", sep=";", encoding='utf8')

In [4]:
def cleanText(tweet):
    if "|" in tweet:
        tweet = tweet.split("|")[1]
        
    tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([0-9])", " ", tweet).split())
    tweet = ' '.join(re.sub("(\w+:\/\/\S+)", " ", tweet).split())
    tweet = ' '.join(re.sub("[\_\|\.\,\"\'\!\?\:\;\$\-\(\)\=]", " ", tweet).split())
    tweet = tweet.lower()
    
    le = list(demoji.findall(tweet))
    for i in le:
        tweet = tweet.replace(i, "")
    
    if tweet.startswith('rt '):
        tweet = tweet.replace("rt ", "")
    
    lNewTweet = []
    for i in tweet.split(" "):
        if i not in palavras_irrelevantes and i not in STOP_WORDS:
            lNewTweet.append(i)
    
    newTweet = " ".join(lNewTweet)
        
    return unidecode.unidecode(newTweet.replace(" rt ", ""))

In [5]:
data["tweet_text_clean"] = data.text.apply(cleanText)

In [6]:
data.head()

,id,user,topic,text,tweet_text_clean
0,"1,31497377565871E+018",DivaDepressao,NaN,RT @PaivaPaulo8: @DivaDepressao pelo amor de D...,amor deus analisa festa
1,"1,31497371872942E+018",DivaDepressao,NaN,RT @andrecarboni666: meu vício agora é pegar ô...,vicio onibus ouvindo filhos gravida
2,"1,31497369440248E+018",DivaDepressao,NaN,RT @bii_aa: Alô dona @NetflixBrasil a senhora ...,alo dona senhora assistiu video viu hahahahhah...
3,"1,3149736672863E+018",DivaDepressao,NaN,@SandroArturBel1 @JornalOGlobo Ihhhhhh,ihhhhhh
4,"1,31497360792013E+018",DivaDepressao,NaN,RT @fatbarbiemarley: @FilhosDaGravida o álcool...,alcool planta edu fala arnica


In [7]:
todas_palavras = ' '.join([texto for texto in data["tweet_text_clean"]])

token_espaco = tokenize.WhitespaceTokenizer()
token_frase = token_espaco.tokenize(todas_palavras)

token_frase_2 = []

for i in token_frase:
    a = cleanText(i)
    if a:
        token_frase_2.append(cleanText(i))


In [8]:
frequencia = nltk.FreqDist(token_frase_2)
freqmc = frequencia.most_common(10000)

# freqmc

In [9]:
#Test Gemsin bigrams
# token_frase_2
# lista_texto = [texto[0] for texto in token_frase_2]
# lista_texto
# # # Build the bigram and trigram models
# t = data["tweet_text_clean"].tolist()
# t

token_ = [doc.split(" ") for doc in data["tweet_text_clean"].tolist()]

# # token_ = [todas_palavras]
import gensim
from gensim.models.phrases import Phraser

bigram = gensim.models.Phrases(token_, min_count=1, threshold=0.1) # higher threshold fewer phrases
trigram = gensim.models.Phrases(bigram[token_], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

l = [trigram_mod[bigram_mod[doc]] for doc in token_]

In [10]:

# dir(bigram_mod)
# # l[6000:]
import gensim.corpora as corpora
id2word = corpora.Dictionary(l)

In [11]:
corpus = [id2word.doc2bow(text) for text in l]

In [12]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [13]:
from pprint import pprint# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"covid" + 0.008*"elenco" + 0.007*"perfeito" + 0.006*"nome" + '
  '0.006*"brasileira" + 0.006*"temporada" + 0.006*"chegou" + 0.006*"play" + '
  '0.005*"alta" + 0.005*"domingo"'),
 (1,
  '0.024*"brasil" + 0.008*"melhor" + 0.007*"calma" + 0.006*"chega_dia" + '
  '0.005*"pandemia" + 0.005*"milhoes" + 0.005*"debate" + 0.005*"amor" + '
  '0.005*"mostra" + 0.004*"livro"'),
 (2,
  '0.007*"vida" + 0.007*"rio" + 0.006*"fala" + 0.006*"natal" + 0.006*"onu" + '
  '0.005*"pt" + 0.005*"dias" + 0.005*"viu" + 0.005*"feito" + 0.005*"tv"'),
 (3,
  '0.014*"governo" + 0.009*"historia" + 0.007*"policia" + 0.007*"acao" + '
  '0.006*"queimadas" + 0.006*"candidatos" + 0.006*"disse" + 0.005*"pantanal" + '
  '0.005*"musica" + 0.005*"twitter"'),
 (4,
  '0.015*"eua" + 0.009*"presidente" + 0.007*"chega" + 0.006*"donald_trump" + '
  '0.005*"ministro" + 0.005*"o" + 0.004*"manha" + 0.004*"amanha" + '
  '0.004*"queda" + 0.004*"mercado"'),
 (5,
  '0.009*"outubro" + 0.005*"ai" + 0.005*"hospital" + 0.005*"ho

In [14]:
from gensim.models import CoherenceModel# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=l, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.6500950642994371
